In [1]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
from tqdm import tqdm

In [ ]:
idList = []
for i in range(168):
    idList.append(i)
queue = pd.read_csv('data/queue.csv', index_col='Unnamed: 0')

In [3]:
edge=pd.read_csv('data/SanFrancisco_edges.csv')

In [ ]:
for i in tqdm(range(4468)):
    flag = 0
    p2x = [[] for row in range(168)]
    p2y = [[] for row in range(168)]
    p3x = [[] for row in range(168)]
    p3y = [[] for row in range(168)]
    spd = [[] for row in range(168)]
    func = pd.DataFrame(columns={'length', 'travel_speed', 'edge_len/Cycle', 'distance_per_cycle', '2', '3', 'congestion'}, index = idList)
    for j in range(2145):
        if pd.isnull(queue.loc[i][j]):
            break
        temp = queue.loc[i][j].split(', ')
        st = temp[0].split(' ')       
        st2 = temp[1].split(' ')
        spd[(int(temp[3]) * 24) + int(temp[4])].append(int(temp[2]))
        p2x[(int(temp[3]) * 24) + int(temp[4])].append(int(st[1]))
        p2y[(int(temp[3]) * 24) + int(temp[4])].append(float(st[0]))
        p3x[(int(temp[3]) * 24) + int(temp[4])].append(int(st2[1]))
        p3y[(int(temp[3]) * 24) + int(temp[4])].append(float(st2[0]))
    for k in range(168):
        if len(p2x[k]) == 0 or len(p2x[k]) == 1:
            continue
        else:
            n = np.polyfit(p2x[k], p2y[k], 1)
            a = round(n[0], 6)
            b = round(n[1], 6)

            n = np.polyfit(p3x[k], p3y[k], 1)
            c = round(n[0], 6)
            d = round(n[1], 6)
            
            if a - c != 0:
                le = a * ((d - b) / (a - c)) + b
                if le > 0:
                    cycle = (sum(p3x[k]) - sum(p2x[k])) / len(p2x[k])
                    # 155초 미만 30초
                    if cycle < 155:
                        cycle = 30
                    # 155이상 185 미만 50초
                    elif cycle >= 155 and cycle < 185:
                        cycle = 50
                    else: # 185 이상은 좌회전 신호가 따로 있을것이므로 40초
                        cycle = 40
                    flag = 1
                    func.loc[k]['length'] = le
                    func.loc[k]['travel_speed'] = edge.loc[i]['length'] / (sum(spd[k]) / len(spd[k]))
                    func.loc[k]['edge_len/Cycle'] = edge.loc[i]['length'] / cycle
                    func.loc[k]['distance_per_cycle'] = func.loc[k]['travel_speed'] * cycle
                    if func.loc[k]['edge_len/Cycle'] > func.loc[k]['travel_speed']:
                        func.loc[k]['2'] = 1
                    else:
                        func.loc[k]['2'] = 0
                    if func.loc[k]['length'] > func.loc[k]['distance_per_cycle']:
                        func.loc[k]['3'] = 1
                    else:
                        func.loc[k]['3'] = 0
                    
                    if func.loc[k]['2'] + func.loc[k]['3'] == 2:
                        func.loc[k]['congestion'] = 1
                    else:
                        func.loc[k]['congestion'] = 0
    func.to_csv('data/queue/queueLen_' + str(i) + '.csv')